In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

# Define image dimensions and batch size
image_size = (480, 640)
batch_size = 64

# Define paths to train and test data directories
train_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/train_data"
test_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/test_data"
valid_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/validation_data"

# Define the number of classes
num_classes = 12

# Create train and test data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(480, 640),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(480,640),
    batch_size=batch_size,
    class_mode='categorical')



# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.001),input_shape=(480,640,3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])


Found 1017 images belonging to 12 classes.
Found 1002 images belonging to 12 classes.
Found 4331 images belonging to 12 classes.


In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 239, 319, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 118, 158, 64)     0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 116, 156, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 58, 78, 128)      0

In [5]:
from keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001, 
    patience=3, 
    verbose=1, 
    mode='auto',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[earlystop_callback]
)



Epoch 1/40
15/15 [==============================] - 45s 3s/step - loss: 5.8693 - accuracy: 0.1039 - val_loss: 2.6117 - val_accuracy: 0.0990
Epoch 2/40
15/15 [==============================] - 33s 2s/step - loss: 2.4399 - accuracy: 0.1721 - val_loss: 2.2612 - val_accuracy: 0.2427
Epoch 3/40
15/15 [==============================] - 37s 3s/step - loss: 2.0092 - accuracy: 0.3893 - val_loss: 1.8909 - val_accuracy: 0.3990
Epoch 4/40
15/15 [==============================] - 33s 2s/step - loss: 1.5160 - accuracy: 0.5362 - val_loss: 2.0722 - val_accuracy: 0.3760
Epoch 5/40
15/15 [==============================] - 32s 2s/step - loss: 1.2039 - accuracy: 0.5939 - val_loss: 1.5067 - val_accuracy: 0.5115
Epoch 6/40
15/15 [==============================] - 33s 2s/step - loss: 0.8886 - accuracy: 0.7125 - val_loss: 1.6574 - val_accuracy: 0.5406
Epoch 7/40
15/15 [==============================] - 33s 2s/step - loss: 0.6582 - accuracy: 0.8080 - val_loss: 1.7304 - val_accuracy: 0.5375
Epoch 8/40
15/15 [==